In [1]:
from get_matrix_from_cnn import *
from get_notes_and_image_paths import *
from get_word_embeddings import *
from image_feature_dataset import *
from load_glove_840B_300d import *
from make_study_dictionary import *

In [52]:
test_dict = main_get_notes_and_image_paths("test")

In [67]:
words = main_get_word_embeddings()

Loading Glove Model
Done. 23094  words loaded!


In [53]:
import torch
import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import math
import torch.optim as optim
import spacy

In [82]:
class TransformerModel(nn.Module):
    """Container module with a linear embedding, positional encoder, encoder, and a decoder."""

    def __init__(self, ndims=1024, wdim=300, vsize=23100, src_pad_inx, pdropout=0.3, device, num_heads, num_encoder_layers, num_decoder_layers, forward_expansion):
        super(TransformerModel, self).__init__()
        self.positional_encoding_densenet = PositionalEncoding(wdim, dropout=0.1)
        self.positional_encoding_chexpert = PositionalEncoding(wdim, dropout=0.1)
        self.embedding_encoder_densenet = LinearEncoding(ndims, wdim, pdropout)
        self.embedding_encoder_chexpert = LinearEncoding(ndims, wdim, pdropout)
        self.positional_decoding = PositionalEncoding(wdim, dropout=0.1)
        self.embedding_decoder = nn.Embedding(vocab_size, wdim)
        self.transformer = nn.Transformer(
            wdim, num_heads, num_encoder_layers, num_decoder_layers, forward_expansion, dropout
        )
        self.fc_out = nn.Linear(wdim, vsize)
        self.dropout = nn.Dropout(pdropout)

    def forward(self, src, trg):
        


SyntaxError: unexpected EOF while parsing (<ipython-input-82-956a5b7f65c3>, line 19)

In [ ]:
class LinearEncoding(nn.Module):
    def __init__(self, ndims, embedding_size=300, dropout_p=0.3):
        super(LinearEncoding, self).__init__()
        self.dropout_p = dropout_p
        
        # Layers
        self.fc1 = nn.Linear(ndims, embedding_size)
        self.dropout = nn.Dropout(dropout_p)
        
    def forward(self, x):
        """Inputs of forward function
        Args:
            x: the sequence fed to the linear encoder model (required)
        Shape of x: (length of sequence for example, number of examples, ndims)
            output: (length of sequence for example, number of examples, embedding_size)

        """
        x = self.fc1(x)
        x = self.dropout(x)
        
        return x

In [ ]:
class PositionalEncoding(nn.Module):
    r"""Inject some information about the relative or absolute position of the tokens
        in the sequence. The positional encodings have the same dimension as
        the embeddings, so that the two can be summed. Here, we use sine and cosine
        functions of different frequencies.
    .. math::
        \text{PosEncoder}(pos, 2i) = sin(pos/10000^(2i/d_model))
        \text{PosEncoder}(pos, 2i+1) = cos(pos/10000^(2i/d_model))
        \text{where pos is the word position and i is the embed idx)
    Args:
        d_model: the embed dim (required).
        dropout: the dropout value (default=0.1).
        max_len: the max. length of the incoming sequence (default=5000).
    Examples:
        >>> pos_encoder = PositionalEncoding(d_model)
    """

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        r"""Inputs of forward function
        Args:
            x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        Examples:
            >>> output = pos_encoder(x)
        """

        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)